In [391]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [392]:
# Features
# X = ['GDPPerCapita', 'CostOfSolar', 'CostOfBatt', 'Population]
# Y = Solar MW Installed

In [393]:
techcostvar = "Solar_Generation_Cost_USD_MW"
battscenario = "Mid_Bat_Cost_USD_MW"

In [394]:
years = ["2000","2012","2013","2014","2015","2016","2017","2018","2019"]

Popfp = '.\Data\PopAndGDP\Population.csv'
Pop = pd.read_csv(Popfp, encoding = "utf-8")
Pop = Pop.melt(id_vars=["Country", "Country Code"], var_name="Year", value_name="population")
Pop["Year"] = Pop["Year"].apply(pd.to_numeric, errors='coerce')

GDPfp = '.\Data\PopAndGDP\GDPPerCapita.csv'
GDP = pd.read_csv(GDPfp, encoding = "utf-8")
GDP = GDP.melt(id_vars=["Country", "Country Code"], var_name="Year", value_name="GDP_Per_Capita")
GDP["Year"] = GDP["Year"].apply(pd.to_numeric, errors='coerce')

TechCostfp = '.\Data\SolarWindConstructionCost\SolarWindConstructionCost.csv'
TechCost = pd.read_csv(TechCostfp, encoding="utf-8")
TechCost = TechCost.apply(pd.to_numeric, errors='coerce')

SolarCost = TechCost[["Year", techcostvar]]
BattCost = TechCost[["Year", battscenario]]

Installedfp = '.\Data\InstalledCapacity\RECAP_20220519-053554.csv'
Installed = pd.read_csv(Installedfp, encoding = "ISO-8859-1")
Installed = Installed[Installed.Technology == "Solar"]
Installed = Installed.melt(id_vars=["Country", "Technology"], var_name="Year", value_name="Installed_Solar_Capacity")
Installed["Year"] = Installed["Year"].apply(pd.to_numeric, errors='coerce')

In [395]:
df = pd.merge(Pop, GDP)
df = df.merge(SolarCost, on="Year")
df = df.merge(BattCost, on="Year")
df = df.merge(Installed, on=["Country", "Year"])
df.head()


,Country,Country Code,Year,population,GDP_Per_Capita,Solar_Generation_Cost_USD_MW,Mid_Bat_Cost_USD_MW,Technology,Installed_Solar_Capacity
0,Afghanistan,AFG,2012,31161378,638.8458516,335.66,640,Solar,16.122
1,Albania,ALB,2012,2900401,4247.630047,335.66,640,Solar,0.679
2,Algeria,DZA,2012,37383899,5592.220115,335.66,640,Solar,25
3,Andorra,AND,2012,82427,38686.46126,335.66,640,Solar,0
4,Angola,AGO,2012,25107925,5100.097027,335.66,640,Solar,9.604


In [396]:
X_params = ["population", "GDP_Per_Capita",techcostvar,battscenario]
Y_param = ["Installed_Solar_Capacity"]

df.replace('..', np.nan, inplace=True)
df = df[X_params + Y_param].dropna()
#df.to_csv("traindataset.csv")


In [397]:

X = df[X_params]
X = X.apply(pd.to_numeric, errors='coerce')
Y = df[Y_param]
Y = Y.apply(pd.to_numeric, errors='coerce')


In [398]:
X.to_csv("Xtraindataset.csv")
Y.to_csv("Ytraindataset.csv")


https://www.weforum.org/agenda/2021/11/renewable-energy-cost-fallen/#:~:text=Back%20in%202010%2C%20a%20megawatt,been%20applicable%20in%20some%20areas.
https://www.windpowermonthly.com/article/1713420/windeconomics-fall-fall-wind-solar-costs
https://www.nrel.gov/docs/fy21osti/79236.pdf

In [399]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

In [400]:
print("X_train shape:" , X_train.shape, "X_test shape:", X_test.shape)

X_train shape: (927, 4) X_test shape: (457, 4)


In [409]:
type(y_train[])

pandas.core.frame.DataFrame

In [402]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='softmax')
])

In [403]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [410]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


C:\Program Files\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\rayno\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\rayno\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\rayno\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\rayno\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py", line 199, 